In [2]:
import openpyxl  ### 엑셀 다루기 위한 패키지
import random
from openpyxl import Workbook, load_workbook
# from kobert_transformers import get_tokenizer
from kogpt2_transformers import get_kogpt2_tokenizer

In [3]:
###### 웰니스 데이터 가공 #####

In [4]:
### (웰니스) 카테고리(구분) / 질문(유저) 데이터 셋 ###

root_path = "../data"
# 데이터셋을 저장할 기본 폴더
wellness_file = root_path + "/wellness_dialog_dataset.xlsx"
# 불러올 파일명
wellness_q_output = root_path + "/wellness_dialog_question_dongbin.txt"
# 정제 후 저장할 파일명
f = open(wellness_q_output, 'w', encoding='utf-8')
# 'w' 쓰기로 파일 열기
wb = load_workbook(filename=wellness_file)
# 엑셀파일 열어서 wb 변수에 저장.

ws = wb[wb.sheetnames[0]]
# 해당 엑셀 파일의 0번째 sheet를 ws에 저장.
for row in ws.iter_rows():
    # iter_rows() 함수로 행 단위 읽기

    f.write(row[0].value + "    " + row[1].value + "\n")

    # 코드 예시
    # 감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.
    # 감정/감정조절이상    더 이상 내 감정을 내가 컨트롤 못 하겠어.
    # row[0].value    + "   " + row[1].value + "\n"

f.close()

In [5]:
### (웰니스) 카테고리(구분) / 답변(챗봇) 데이터 셋 ###

root_path = "../data"
wellness_file = root_path + "/wellness_dialog_dataset.xlsx"
wellness_a_output = root_path + "/wellness_dialog_answer_dongbin.txt"

f = open(wellness_a_output, 'w', encoding='utf-8')
wb = load_workbook(filename=wellness_file)
ws = wb[wb.sheetnames[0]]

for row in ws.iter_rows():
    if row[2].value == None:  # 엑셀 파일 컬럼C열(챗봇)에 데이터가 없는 경우도 있다.
        continue
    else:
        f.write(row[0].value + "    " + row[2].value + "\n")

        # 감정/감정조절이상    감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
        # 감정/감정조절이상    저도 그 기분 이해해요. 많이 힘드시죠?

f.close()

In [6]:
### (웰니스) 질문(유저) / 답변(챗봇) 데이터 셋 ###
root_path = "../data"
# wellness_file = root_path + "/wellness_dialog_dataset.xlsx"
wellness_answer_file = root_path + "/wellness_dialog_answer_dongbin.txt"
wellness_question_file = root_path + "/wellness_dialog_question_dongbin.txt"
# 위 두 파일에서 질문과 답변을 추출해서 아래의 파일명에 저장한다.
wellness_autoregressive_file = root_path + "/wellness_dialog_for_autoregressive_dongbin.txt"
# 질문/답변 셋이 저장될 파일명

answ_file = open(wellness_answer_file, 'r', encoding='utf-8')
ques_file = open(wellness_question_file, 'r', encoding='utf-8')
# 파일 열기
autoregressive_file = open(wellness_autoregressive_file, 'w', encoding='utf-8')
# 정제된 데이터가 저장될 파일 열기
answ_lines = answ_file.readlines()
ques_lines = ques_file.readlines()

ques_dict = {}
for line_num, line_data in enumerate(ques_lines):
    #     print(line_num, line_data)
    #     1 감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.
    #     2 감정/감정조절이상    더 이상 내 감정을 내가 컨트롤 못 하겠어.

    ques_data = line_data.split('    ')
    #     print(ques_data)
    #     ['감정/감정조절이상', '제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.\n']
    #     ['감정/감정조절이상', '더 이상 내 감정을 내가 컨트롤 못 하겠어.\n']

    for ans_line_num, ans_line_data in enumerate(answ_lines):
        #         print(ans_line_num, ans_line_data)
        #         1 감정/감정조절이상    감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
        #         2 감정/감정조절이상    저도 그 기분 이해해요. 많이 힘드시죠?

        ans_data = ans_line_data.split('    ')
        #         print(ans_data)
        #         ['감정/감정조절이상', '감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.\n']
        #         ['감정/감정조절이상', '저도 그 기분 이해해요. 많이 힘드시죠?\n']

        if ques_data[0] == ans_data[0]:  # 두 파일의 카테고리(구분)이 같으면
            autoregressive_file.write(ques_data[1][:-1] + "    " + ans_data[1])
            # -1은 끝에 \n을 없애기 위함.
        #             제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.    감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
        #             제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.    저도 그 기분 이해해요. 많이 힘드시죠?

        else:
            continue

answ_file.close()
ques_file.close()
autoregressive_file.close()

In [7]:
# ### (웰니스) 질문(유저) / 답변(챗봇) 데이터 셋  +  토큰 적용  ###
root_path = "../data"
wellness_answer_file = root_path + "/wellness_dialog_answer_dongbin.txt"
wellness_question_file = root_path + "/wellness_dialog_question_dongbin.txt"
wellness_autoregressive_file = root_path + "/wellness_dialog_for_autoregressive_with_token_dongbin.txt"

answ_file = open(wellness_answer_file, 'r', encoding='utf-8')
ques_file = open(wellness_question_file, 'r', encoding='utf-8')
autoregressive_file = open(wellness_autoregressive_file, 'w', encoding='utf-8')

answ_lines = answ_file.readlines()
ques_lines = ques_file.readlines()
# ques_dict = {}
for line_num, line_data in enumerate(ques_lines):
    ques_data = line_data.split('    ')
    for ans_line_num, ans_line_data in enumerate(answ_lines):
        ans_data = ans_line_data.split('    ')
        if ques_data[0] == ans_data[0]:
            autoregressive_file.write("<s>" + ques_data[1][:-1] + "</s><s>" + ans_data[1][:-1] + "</s>\n")
        #             print("<s>" + ques_data[1][:-1] + "</s><s>" + ans_data[1][:-1] + "</s>\n")
        #             <s>제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.</s><s>감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.</s>
        #             <s>제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.</s><s>저도 그 기분 이해해요. 많이 힘드시죠?</s>
        else:
            continue

answ_file.close()
ques_file.close()
autoregressive_file.close()

In [8]:
### 카테고리(구분) 클래스 데이터 (총 359개) ###
# def category_data():
root_path = "../data"
data_path = root_path + "/wellness_dialog_dataset.txt"
c_output = root_path + "/wellness_dialog_category_dongbin.txt"

i_f = open(data_path, 'r', encoding='utf-8')
o_f = open(c_output, 'w', encoding='utf-8')

category_count = 0  # 카테고리 종류의 유니크 숫자 부여

cate_dict = []
i_lines = i_f.readlines()
for i, data in enumerate(i_lines):
    tmp = data.split(',')  # txt파일 데이터르 쉼표로 구분하여 리스트화
    # a = tmp[1]
    q = tmp[0]
    if q not in cate_dict and q != '구분':
        cate_dict.append(q)
        o_f.write(q.strip() + "    " + str(category_count) + "\n")
        category_count += 1

#         감정/감정조절이상    0
#         감정/감정조절이상/화    1
#         감정/걱정    2

o_f.close()
i_f.close()

In [9]:
### 질의와 카테고리(구분) 클래스 쌍 데이터 5231 개  ####
# def wellness_text_classification_data():

root_path = "../data"
wellness_category_file = root_path + "/wellness_dialog_category_dongbin.txt"
wellness_question_file = root_path + "/wellness_dialog_question_dongbin.txt"
wellness_text_classification_file = root_path + "/wellness_dialog_for_text_classification_dongbin.txt"

cate_file = open(wellness_category_file, 'r', encoding='utf-8')
ques_file = open(wellness_question_file, 'r', encoding='utf-8')
text_classfi_file = open(wellness_text_classification_file, 'w', encoding='utf-8')

category_lines = cate_file.readlines()
cate_dict = {}
for line_num, line_data in enumerate(category_lines):
    #     감정/감정조절이상    0
    #     감정/감정조절이상/화    1
    #     감정/걱정    2
    data = line_data.split('    ')
    cate_dict[data[0]] = data[1][:-1]

# 딕셔너리를 만들어 준다.
# print(cate_dict)
# {'감정/감정조절이상': '0', '감정/감정조절이상/화': '1', '감정/걱정': '2', ... ... }


ques_lines = ques_file.readlines()
ques_dict = {}
for line_num, line_data in enumerate(ques_lines):
    #     감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.
    #     감정/감정조절이상    더 이상 내 감정을 내가 컨트롤 못 하겠어.
    data = line_data.split('    ')
    try:  # '컬럼명 : 구분, 유저' 때문에.
        text_classfi_file.write(data[1][:-1] + "    " + cate_dict[data[0]] + "\n")

    #         print(data[1][:-1] + "    " + cate_dict[data[0]] + "\n")
    #         제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.    0
    #         더 이상 내 감정을 내가 컨트롤 못 하겠어.    0
    except:
        pass

cate_file.close()
ques_file.close()
text_classfi_file.close()

In [10]:
### 트레인 테스트 셋으로 나누기  ####
### wellness_dialog_for_autoregressive_dongbin.txt 를 테스트 트레인 셋으로 나눔
# def seperate_wellness_data():
import random

file_path = root_path + "/wellness_dialog_for_autoregressive_dongbin.txt"
train_file_path = root_path + "/wellness_dialog_for_autoregressive_train_dongbin.txt"
test_file_path = root_path + "/wellness_dialog_for_autoregressive_test_dongbin.txt"

sperated_file = open(file_path, 'r', encoding='utf-8')
train_file = open(train_file_path, 'w', encoding='utf-8')
test_file = open(test_file_path, 'w', encoding='utf-8')

sperated_file_lines = sperated_file.readlines()

for line_num, line_data in enumerate(sperated_file_lines):
    rand_num = random.randint(0, 10)
    if rand_num < 10:  # 9:1 확률인거신가???
        train_file.write(line_data)
    else:
        test_file.write(line_data)

sperated_file.close()
train_file.close()
test_file.close()